# Capstone Project - The Battle of Neighborhoods

### Notebook for the battle of the Neighborhoods Capstone. We will be exploring the Foursquare API data and how to apply it to a business problem. 

## Business Problem

### We will be exploring the Washington D.C. area in order to assist in informing our client the best location for opening a high-end French restuarant. We will identify a location in D.C. that explores the area for other restaurants, other French restaurants, and the median income of the area and surrounding areas, based on median income data from Census. Our stakeholders include not only our client, but also the customers who will be eating there, and an exploration of available transportation in the area will also be explored.

## Data 

### The three following data sets will be used to arrive at a conclusino for the best location for a high-end Frech restaurant based on competition, median income, and available transportation in the area:

### Foursquare Data: We will use the Foursquare API to explore French restaurant locations in D.C. and pick a location that has little competition for out stakeholder (client).

### Census Data: We will use 2010 Census data in order to find the median income by Census block, and use this to find the areas with the highest income, so the restaurant can experience the highest traffic possible given the high price point of the food.

### DC Open Data Transporation: We will use the DC Open Data portal in order to find transportation in the area that will make it easier for one of our stakeholders (customers) to get there.

## Data Wrangling

### Foursquare

In [1]:
import requests 
import pandas as pd
import numpy as np 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium

import config as keys

# Foursquare API info
CLIENT_ID = keys.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = keys.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180604'

In [2]:
dc_center = (38.942900, -77.030364)

latitude, longitude = dc_center[0], dc_center[1]

radius = 5000

categoryId = '4bf58dd8d48988d10c941735' # French restaurant Foursquare ID

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}'.format(CLIENT_ID, 
                                                                                                                               CLIENT_SECRET, 
                                                                                                                               latitude, 
                                                                                                                               longitude, 
                                                                                                                               VERSION, 
                                                                                                                               categoryId, 
                                                                                                                               radius)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
french_venues = pd.json_normalize(venues)
french_df = french_venues.drop(['id', 'categories', 'referralId', 'hasPerk', 'location.labeledLatLngs', 'location.formattedAddress', 'delivery.id', 'delivery.url', 'delivery.provider.name', 'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes', 'delivery.provider.icon.name', 'venuePage.id'], axis=1)
french_df.head()

,name,location.address,location.crossStreet,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.city,location.state,location.country
0,Opaline,806 15th St NW,btwn H & I St NW,38.900498,-77.033970,4730,20005,US,Washington,D.C.,United States
1,Le Zinc,3714 Macomb St NW,btwn Wisconsin Ave & 37th St,38.939434,-77.078279,4166,20016,US,Washington,D.C.,United States
2,Un Je Ne Sais Quoi,1361 Connecticut Ave NW,NaN,38.908752,-77.042634,3947,20036,US,Washington,D.C.,United States
3,Bistro Cacao,320 Massachusetts Ave NE,NaN,38.894417,-77.001033,5965,20002,US,Washington,D.C.,United States
4,Bistrot Lepic & Wine Bar,1736 Wisconsin Ave NW,at S St NW,38.914606,-77.067265,4486,20007,US,Washington,D.C.,United States


In [3]:
# Load in data - API URL JSON
metro_map = folium.Map(location=[38.932645,-76.984256], 
                       zoom_start = 12, 
                       tiles = keys.mapbox_tile,
                       attr = 'Navigation'
                       )

for label, lat, lng in zip(french_df['name'], french_df['location.lat'], french_df['location.lng']):
    folium.Marker(
        [lat, lng],
        radius=5,
        color='red',
        icon=folium.Icon(color='green'),
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(metro_map)

metro_map

### DC Open Data - Metro Stations

In [4]:
# Load in data - API URL GeoJSON
metro_url = 'https://opendata.arcgis.com/datasets/ab5661e1a4d74a338ee51cd9533ac787_50.geojson'

folium.GeoJson(metro_url, name='geojson').add_to(metro_map)

metro_map

### Census - Median Incomes by Census Tract

In [6]:
# API URL GeoJSON

median_url = 'https://opendata.arcgis.com/datasets/0ccdf1f2d4fe4a5391899c595fd2e02c_0.geojson'

median_data = pd.read_csv('dc_data/median.csv')

bins = list(median_data['Median_Income'].quantile([0, 0.25, 0.5, 0.75, 1]))

# folium.GeoJson(median_url, name='geojson').add_to(metro_map)

folium.Choropleth(geo_data = median_url,
                   name = 'median_chloro',
                   data = median_data,
                   columns = ['NAME', 'Median_Income'],
                   key_on = 'feature.properties.NAME',
                   fill_color = 'PuBuGn',
                   fill_opacity = 0.7,
                   line_opacity = 0.5,
                   legend_name = 'Median Income Past 12 Months',
                   bins = bins).add_to(metro_map)

metro_map

### Uber - Travel times to locations